# HOW-TO-SOFTWARE-5.0-PAPERS

Quest: Software 5.0 Scrolls
Belt target: White -> Green

Objective:
- verify papers are first-class runtime assets
- verify books + papers are both discoverable
- verify twin CPU prepass exposes `/papers`


In [ ]:
from pathlib import Path
import json
import subprocess

repo = Path.cwd().resolve()
while repo != repo.parent and not (repo / 'cli' / 'src').exists():
    repo = repo.parent
assert (repo / 'cli' / 'src').exists(), 'Repo root not found'

env_prefix = 'PYTHONPATH=' + str(repo / 'cli/src')

checks = [
    'papers list --json',
    'papers show 00-index',
    'books list --json',
]

for args in checks:
    cmd = env_prefix + ' python -m stillwater ' + args
    print('$ ' + cmd)
    proc = subprocess.run(['bash', '-lc', cmd], cwd=str(repo), capture_output=True, text=True)
    print(proc.stdout)
    if proc.returncode != 0:
        print(proc.stderr)
        raise RuntimeError('Command failed: ' + args)

twin_cmd = env_prefix + ' python -m stillwater twin "/papers" --json'
print('$ ' + twin_cmd)
twin = subprocess.run(['bash', '-lc', twin_cmd], cwd=str(repo), capture_output=True, text=True)
print(twin.stdout)
if twin.returncode != 0:
    print(twin.stderr)
    raise RuntimeError('Twin /papers failed')

payload = json.loads(twin.stdout)
assert payload.get('source') == 'CPU', 'Expected CPU prepass route for /papers'
